In [1]:
import pandas as pd
import numpy as np
import pyproj as pj

In [2]:
fsp = pd.read_csv('../data/fault_data/hayes_finite_fault.fsp', 
                  delim_whitespace=True, comment='%', skiprows=50,
                  names=['lat', 'lon', 'x', 'y', 'z', 'slip_m', 'rake', 't_rup', 'rise'])

In [3]:
fsp.head()

,lat,lon,x,y,z,slip_m,rake,t_rup,rise
0,26.9545,85.8867,112.0661,-140.8269,1.9764,0.1815,75.1493,107.6,9.6
1,27.0305,85.7017,93.6608,-132.5311,1.9764,0.1372,78.2388,98.4,11.2
2,27.1066,85.5166,75.2604,-124.1967,1.9764,0.1409,109.2604,87.2,11.2
3,27.1827,85.3316,56.8998,-115.8346,1.9764,0.0012,105.4416,74.4,1.6
4,27.2587,85.1466,38.5593,-107.4561,1.9764,0.0216,125.1266,60.4,1.6


In [5]:
fsp['strike'] = 295.
fsp['dip'] = 10.
fsp['fault_name'] = 'MHT_rupture'


In [6]:
fsp.head()

,lat,lon,x,y,z,slip_m,rake,t_rup,rise,strike,dip,fault_name
0,26.9545,85.8867,112.0661,-140.8269,1.9764,0.1815,75.1493,107.6,9.6,295,10,MHT_rupture
1,27.0305,85.7017,93.6608,-132.5311,1.9764,0.1372,78.2388,98.4,11.2,295,10,MHT_rupture
2,27.1066,85.5166,75.2604,-124.1967,1.9764,0.1409,109.2604,87.2,11.2,295,10,MHT_rupture
3,27.1827,85.3316,56.8998,-115.8346,1.9764,0.0012,105.4416,74.4,1.6,295,10,MHT_rupture
4,27.2587,85.1466,38.5593,-107.4561,1.9764,0.0216,125.1266,60.4,1.6,295,10,MHT_rupture


In [7]:
htm = pd.read_csv('../data/fault_data/htm_faults.csv')

In [8]:
htm.tail()

,east,north,depth,point_index,strike,dip,rake,fault_name,htm_ind
1595,333071.108041,3195863.464197,-8696,175,194.926051,55,-90,NaN,922
1596,332868.103509,3197976.078167,-8605,176,194.926051,55,-90,NaN,922
1597,332900.874850,3200349.175353,-8676,177,194.926051,55,-90,NaN,922
1598,333508.166327,3202318.759937,-8796,178,194.926051,55,-90,NaN,922
1599,334474.029461,3205074.635366,-8490,179,194.926051,55,-90,NaN,922


In [9]:
htm['slip_m'] = 0.05

In [10]:
ff = fsp[['z', 'slip_m', 'rake', 'strike', 'dip']]

#ff.columns=['east', 'north', 'depth', 'slip_m', 'rake', 'strike', 'dip']

In [11]:
wgs84 = pj.Proj(init='epsg:4326')
utm45 = pj.Proj(init='epsg:32645')

ff['east'], ff['north'] = pj.transform(wgs84, utm45, fsp.lon.values, fsp.lat.values)

In [12]:
ff.head()

,z,slip_m,rake,strike,dip,east,north
0,1.9764,0.1815,75.1493,295,10,389495.184377,2981882.397859
1,1.9764,0.1372,78.2388,295,10,371217.176384,2990476.582043
2,1.9764,0.1409,109.2604,295,10,352953.546302,2999109.728820
3,1.9764,0.0012,105.4416,295,10,334724.026010,3007770.701491
4,1.9764,0.0216,125.1266,295,10,316518.527897,3016448.681931


In [13]:
ff = ff[['east', 'north', 'z', 'strike', 'dip', 'slip_m', 'rake']]
ff.columns = ['east', 'north', 'depth', 'strike', 'dip', 'slip_m', 'rake']

In [14]:
ff['fault_name'] = 'MHT_rupture'
ff['point_index'] = ff.index.values
ff['htm_ind'] = 9999

In [37]:
ff.depth *= -1000

In [41]:
faults = pd.concat([htm, ff])

In [42]:
faults.shape

(1721, 10)

In [43]:
faults.head()

,depth,dip,east,fault_name,htm_ind,north,point_index,rake,slip_m,strike
0,4623,55,210702.346219,NaN,30,3295884.106254,0,-90,0.1,19.761917
1,4762,55,205435.348084,NaN,30,3287130.049210,1,-90,0.1,19.761917
2,4831,55,200405.436713,NaN,30,3267428.950486,2,-90,0.1,19.761917
3,4677,55,200853.134701,NaN,30,3259918.710871,3,-90,0.1,19.761917
4,4797,55,198732.170284,NaN,30,3242085.302656,4,-90,0.1,19.761917


In [44]:
faults = faults[['east', 'north', 'depth', 'strike', 'dip', 'slip_m', 'rake',
                 'fault_name', 'point_index', 'htm_ind']]

In [45]:
faults.to_csv('../data/fault_data/all_fault_pts.csv')